In [2]:
%matplotlib inline
import torch
from torch.utils.data import TensorDataset, DataLoader  
from torch.nn import init
import torch.optim as optim 
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from collections import OrderedDict
import numpy as np
import time 
from IPython import display
from matplotlib import pyplot as plt

In [3]:
def dropout(X, drop_prob):
    X = X.float()
    assert 0<=drop_prob<=1
    keep_prob = 1- drop_prob
    if keep_prob == 0:
        return torch.zeros_like(X)
    mask = (torch.rand(X.shape,dtype = torch.float)<keep_prob).float()
    return mask*X/keep_prob

In [4]:
X = torch.arange(16).view(2,8)

In [5]:
num_inputs, num_outputs, num_hiddens1, num_hiddens2 = 784, 10, 256, 256

w1 = torch.tensor(np.random.normal(0, 0.01, (num_inputs, num_hiddens1)), dtype = torch.float,requires_grad = True)
b1 = torch.zeros(num_hiddens1, requires_grad = True)
w2 = torch.tensor(np.random.normal(0, 0.01, (num_hiddens1, num_hiddens2)), dtype = torch.float,requires_grad = True)
b2 = torch.zeros(num_hiddens2, requires_grad = True)
w3 = torch.tensor(np.random.normal(0, 0.01, (num_hiddens2, num_outputs)), dtype = torch.float,requires_grad = True)
b3 = torch.zeros(num_outputs, requires_grad = True)

params = [w1, b1, w2, b2, w3, b3]

In [6]:
#数据切割
def load_data_fashion_mnist(batch_size):
    mnist_train = torchvision.datasets.FashionMNIST(root = './Dataset/FashionMNIST', train = True, 
                                                    download = True, transform = transforms.ToTensor())
    mnist_test = torchvision.datasets.FashionMNIST(root = './Dataset/FashionMNIST', train = False, 
                                                   download = True, transform = transforms.ToTensor())
    num_workers = 4
    train_iter = DataLoader(mnist_train, batch_size = batch_size, shuffle = True, num_workers = num_workers)
    test_iter = DataLoader(mnist_test, batch_size = batch_size, shuffle = True, num_workers = num_workers)
    return train_iter, test_iter

train_iter, test_iter = load_data_fashion_mnist(batch_size = 256)

In [7]:
def network(X, params, drop_prob1, drop_prob2, is_training = True ):
    X = X.view(-1, num_inputs)
    w1, b1, w2, b2, w3, b3 = params
    h1 = (torch.matmul(X, w1) + b1).relu()
    if is_training:
        h1 = dropout(h1, drop_prob1)
    h2 = (torch.matmul(h1, w2) + b2).relu()
    if is_training:
        h2 = dropout(h2, drop_prob2)
    return torch.matmul(h2, w3) + b3    

In [8]:
def softmax(y):
    y_exp = y.exp()
    partition = y_exp.sum(dim = 1, keepdim = True)
    return (y_exp/partition)

In [9]:
def sgd(params, lr):
    for param in params:
        param.data -= lr*param.grad

In [10]:
def crossentrpyloss(y_hat, y):
    return -(torch.log(y_hat.gather(1, y.view(-1,1))).mean())

In [26]:
def evaluate_accuracy(data_iter, network, params = None, drop_prob1 = 0.2, drop_prob2 = 0.5):
    acc_num, n = 0, 0
    for X, y in data_iter:
        if isinstance(network, nn.Module):
            network.eval()
            acc_num += (network(X).argmax(dim = 1, keepdim = True) == y.view(-1,1)).float().sum().item()
            network.train()
        else:
            if('is_training' in network.__code__.co_varnames):
                acc_num += (network(X, params, drop_prob1, drop_prob2, is_training = False).argmax(dim = 1, keepdim = True) == y.view(-1,1)).float().sum().item()
            else:
                acc_num += (network(X).argmax(dim = 1, keepdim = True) == y.view(-1,1)).float().sum().item()
        n += len(y)
    return acc_num/n

In [14]:
def train_loop_dropout(n_epoch, drop_prob1, drop_prob2, 
                       train_iter, test_iter, 
                       model_fn, loss_fn, 
                       params = None, lr = None, optimizer = None):
    for epoch in range(n_epoch):
        train_loss, train_acc_num, n, i = 0.0, 0.0, 0, 0
        for train_X, train_y in train_iter:
            y_hat = softmax(model_fn(train_X, params, drop_prob1, drop_prob2))
            loss = loss_fn(y_hat, train_y)
            
            if optimizer is not None:
                optimizer.zero_grad()
            elif params is not None and params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            
            loss.backward()
            
            if optimizer is not None:
                optimizer.step()
            else:
                sgd(params, lr)
            
            train_loss += loss.item()
            train_acc_num += (y_hat.argmax(dim = 1) == train_y).float().sum().item()
            n += len(train_y)
            i+=1
        if epoch%5 == 0:
            test_loss, j = 0.0, 0
            acc_test = evaluate_accuracy(test_iter, params, drop_prob1, drop_prob2, model_fn)
            acc_train = train_acc_num/n
            for test_X, test_y in test_iter:
                j += 1
                test_loss += loss_fn(softmax(model_fn(test_X, params, drop_prob1, drop_prob2, is_training = False)), test_y).item()
            print('epoch: %d, acc_train: %.2f, acc_test: %.2f, train_loss: %.2f, test_loss, %.2f'% (epoch, acc_train, acc_test, train_loss/i, test_loss/j))
    #return params

In [15]:
train_loop_dropout(n_epoch = 20, drop_prob1 = 0.2, drop_prob2 = 0.5, 
                       train_iter = train_iter, test_iter = test_iter, 
                       model_fn = network, loss_fn = crossentrpyloss, 
                       params = params, lr = 0.1, optimizer = None)

epoch: 0, acc_train: 0.88, acc_test: 0.87, train_loss: 0.33, test_loss, 0.36
epoch: 5, acc_train: 0.89, acc_test: 0.85, train_loss: 0.31, test_loss, 0.43
epoch: 10, acc_train: 0.89, acc_test: 0.87, train_loss: 0.29, test_loss, 0.37
epoch: 15, acc_train: 0.90, acc_test: 0.88, train_loss: 0.28, test_loss, 0.34


In [16]:
class FlattenLayer(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, X):
        return X.view(X.shape[0],-1)

In [18]:
drop_prob1, drop_prob2 = 0.2, 0.5

In [20]:
network = nn.Sequential(FlattenLayer(),
                        nn.Linear(num_inputs, num_hiddens1),
                        nn.ReLU(),
                        nn.Dropout(drop_prob1),
                        nn.Linear(num_hiddens1, num_hiddens2),
                        nn.ReLU(),
                        nn.Dropout(drop_prob2),
                        nn.Linear(num_hiddens2, num_outputs))

for param in network.parameters():
    nn.init.normal_(param, mean = 0, std = 0.01)

optimizer = optim.SGD(network.parameters(), lr = 0.1)

In [28]:
def train_loop_dropout_nn(n_epoch, train_iter, test_iter, 
                       model_fn, loss_fn, 
                       params = None, lr = None, optimizer = None):
    for epoch in range(n_epoch):
        train_loss, train_acc_num, n, i = 0.0, 0.0, 0, 0
        for train_X, train_y in train_iter:
            y_hat = model_fn(train_X)
            loss = loss_fn(y_hat, train_y)
            
            if optimizer is not None:
                optimizer.zero_grad()
            elif params is not None and params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            
            loss.backward()
            
            if optimizer is not None:
                optimizer.step()
            else:
                sgd(params, lr)
            
            train_loss += loss.item()
            train_acc_num += (y_hat.argmax(dim = 1) == train_y).float().sum().item()
            n += len(train_y)
            i+=1
        if epoch%5 == 0:
            test_loss = 0.0
            acc_test = evaluate_accuracy(test_iter, model_fn)
            acc_train = train_acc_num/n
            print('epoch: %d, acc_train: %.2f, acc_test: %.2f, test_loss, %.2f'% (epoch, acc_train, acc_test, train_loss/i))
    #return params

In [29]:
train_loop_dropout_nn(n_epoch = 20, train_iter = train_iter, test_iter = test_iter, 
                       model_fn = network, loss_fn = nn.CrossEntropyLoss(), 
                       params = None, lr = None, optimizer = optimizer)

epoch: 0, acc_train: 0.79, acc_test: 0.76, test_loss, 0.60
epoch: 5, acc_train: 0.85, acc_test: 0.85, test_loss, 0.44
epoch: 10, acc_train: 0.87, acc_test: 0.84, test_loss, 0.38
epoch: 15, acc_train: 0.88, acc_test: 0.85, test_loss, 0.34
